In [2]:
#Import required Libraries. 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cx_Oracle 
import gzip
import calendar
import pickle
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta


#Connection to corporate Server
conn = cx_Oracle.connect('ReportUser/report@10.10.1.7:1521/rproods')
cur = conn.cursor()
sql = "select ACTIVE from CUSTOMER_V"
cur.execute(sql)
resutl = cur.fetchall()


In [3]:
#Last Month Details
#last_month = date.today().strftime("%m")
#Last_Sales_Date = '30/11/2019'

In [4]:
def getLastWeekCustomers(sbs):
    conn = cx_Oracle.connect('ReportUser/report@10.10.1.7:1521/rproods')
    cur = conn.cursor()
    #Dates Needed for SQL Script. 

    today =date.today()
    last_date = date.today() - relativedelta(months=+1)
    last_date = str(last_date.day) +"/" +str(last_date.month)+"/"+str(last_date.year)
    next_monday = today + timedelta(days=-today.weekday(), weeks=1)
    lastMonday = next_monday -timedelta(days=14)
    lastSunday = lastMonday+timedelta(days=6)
    lastMondy = str(lastMonday.day)+"/"+str(lastMonday.month)+"/"+str(lastMonday.year)
    lastSundy = str(lastSunday.day)+"/"+str(lastSunday.month)+"/"+str(lastSunday.year)
    lastWeek = "select count(c.CUST_SID) FROM CUSTOMER_V c WHERE c.LST_SALE_DATE BETWEEN to_date('{}','DD/MM/YYYY') AND to_date('{}','DD/MM/YYYY') AND c.SBS_NO={}".format(lastMondy, lastSundy, sbs)
    cur.execute(lastWeek)
    s =  cur.fetchall()
    lastWeekCustomers = {'Last Week Customers - {}'.format(sbs): s[0][0]}
    return lastWeekCustomers
    


In [5]:
def getSixMonthsCustomers(sbs):
    conn = cx_Oracle.connect('ReportUser/report@10.10.1.7:1521/rproods')
    cur = conn.cursor()
    #Dates Needed for SQL Script. 
    six_months = date.today() - relativedelta(months=+6)
    six_months = str(six_months.day) +"/" +str(six_months.month)+"/"+str(six_months.year)

    #six = "select c.CUST_SID, c.SBS_NO,c.FIRST_NAME, c.LAST_NAME, c.CREATED_DATE,c.LST_SALE_DATE, c.LST_SALE_AMT, a.PHONE1, a.EMAIL_ADDR FROM CUSTOMER_V c LEFT JOIN CUST_ADDRESS_V a ON c.CUST_SID=a.CUST_SID WHERE c.LST_SALE_DATE > to_date('{}','DD/MM/YYYY')".format(six_months)
    six = "select count(c.CUST_SID) FROM CUSTOMER_V c WHERE c.LST_SALE_DATE > to_date('{}','DD/MM/YYYY') and c.SBS_NO = {}".format(six_months,sbs)
    cur.execute(six)
    s =  cur.fetchall()
    sixMonthsCount = {'Last 6 Months Customers - {}'.format(sbs): s[0][0]}
    return sixMonthsCount

In [5]:
def getNewLastWeekCustomers(sbs):
    conn = cx_Oracle.connect('ReportUser/report@10.10.1.7:1521/rproods')
    cur = conn.cursor()
    #Dates Needed for SQL Script. 

    today =date.today()
    last_date = date.today() - relativedelta(months=+1)
    last_date = str(last_date.day) +"/" +str(last_date.month)+"/"+str(last_date.year)
    next_monday = today + timedelta(days=-today.weekday(), weeks=1)
    lastMonday = next_monday -timedelta(days=14)
    lastSunday = lastMonday+timedelta(days=6)
    lastMondy = str(lastMonday.day)+"/"+str(lastMonday.month)+"/"+str(lastMonday.year)
    lastSundy = str(lastSunday.day)+"/"+str(lastSunday.month)+"/"+str(lastSunday.year)
    lastWeek = "select count(c.CUST_SID) FROM CUSTOMER_V c WHERE c.SBS_NO = {} AND  c.CREATED_DATE BETWEEN to_date('{}','DD/MM/YYYY') AND to_date('{}','DD/MM/YYYY')".format(sbs,lastMondy, lastSundy)
    cur.execute(lastWeek)
    s =  cur.fetchall()
    lastWeekCustomers = {'Last Week New Customers - {}'.format(sbs): s[0][0]}
    return lastWeekCustomers


In [6]:
print(getNewLastWeekCustomers(200))

{'Last Week New Customers - 200': 378}


In [7]:
def totalCustomers(sbs):
    conn = cx_Oracle.connect('ReportUser/report@10.10.1.7:1521/rproods')
    cur = conn.cursor()
    #Dates Needed for SQL Script. 
    lastWeek = "select count(CUST_SID) FROM CUSTOMER_V where ACTIVE=1 AND SBS_NO= {} ".format(sbs)
    cur.execute(lastWeek)
    s =  cur.fetchmany()
    lastWeekCustomers = {'Total Customers - {}'.format(sbs): s[0][0]}
    return lastWeekCustomers

In [8]:
print(totalCustomers(200))

{'Total Customers - 200': 139132}


In [9]:
customers = {}
sub = (100,200)

for i in sub:
    customers.update(totalCustomers(i))
    customers.update(getSixMonthsCustomers(i))
    customers.update(getLastWeekCustomers(i))
    customers.update(getNewLastWeekCustomers(i))

In [10]:
customers

{'Total Customers - 100': 548236,
 'Last 6 Months Customers - 100': 132512,
 'Last Week Customers - 100': 7804,
 'Last Week New Customers - 100': 1420,
 'Total Customers - 200': 139132,
 'Last 6 Months Customers - 200': 31479,
 'Last Week Customers - 200': 1610,
 'Last Week New Customers - 200': 378}

In [11]:
data = pd.DataFrame.from_dict(customers, orient='index', columns= ['Count'])

In [12]:
data

,Count
Total Customers - 100,548236
Last 6 Months Customers - 100,132512
Last Week Customers - 100,7804
Last Week New Customers - 100,1420
Total Customers - 200,139132
Last 6 Months Customers - 200,31479
Last Week Customers - 200,1610
Last Week New Customers - 200,378


# code for Transaction Count


In [ ]:
d = [('2/02/2020','8/02/2020',100 ), ('2/02/2019','8/02/2019',200 )]
counts = {}
for i in d:
    query = "select count(invc_sid) as trans_count from invoice_v where sbs_no={} and invc_type=0 and created_date between to_date('{}', 'DD/MM/YYYY') AND to_date('{}', 'DD/MM/YYYY')".format(i[2], i[0], i[1])
    cur.execute(query)
    count =  cur.fetchone()
    sub = "HealthPlus" if i[2] == 100 else "CasaBella"
    
    dates = sub + ': ' + i[0]+' - '+ i[1]
    counts.update({dates: count})
     
x = list(counts.values())
x = [i[0] for i in x]
y =  list(counts.keys())
y = [i.split('-')[0] for i in y]
y = [i.rstrip() for i in y]
plt.plot(y, x)
plt.xlabel = "Week"
plt.ylable = "Trans Coutn"


In [ ]:
counts

In [13]:
data.to_csv('Customers_count.csv')

# End of Script

today =date.today()
last_date = date.today() - relativedelta(months=+1)
last_date = str(last_date.day) +"/" +str(last_date.month)+"/"+str(last_date.year)
next_monday = today + timedelta(days=-today.weekday(), weeks=1)
lastMonday = next_monday -timedelta(days=14)
lastSunday = lastMonday+timedelta(days=6)

last_sale_date = date.today() - relativedelta(months=+1)
last_date = str(last_sale_date.day) +"/" +str(last_sale_date.month)+"/"+str(last_sale_date.year)
print(last_date)

print(getnewCustomers())